In [1]:
import numpy as np
from pathlib import Path

splits_dir = Path("../data/splits")

val   = np.load(splits_dir / "val.npz",   allow_pickle=True)
test  = np.load(splits_dir / "test.npz",  allow_pickle=True)

X_val   = val["X"].astype(np.float32)
y_val   = val["y"].astype(int)

X_test  = test["X"].astype(np.float32)
y_test  = test["y"].astype(int)

# Feature names need allow_pickle=True
feature_columns = np.load(
    splits_dir / "feature_columns.npy",
    allow_pickle=True
).tolist()

In [2]:
cols_to_remove = ["isTimelineWork", "isPublicDomain", "accessionYear"]
# cols_to_remove = ["isTimelineWork", "isPublicDomain", "accessionYear"]

# Get indices of columns we want to drop
drop_indices = [feature_columns.index(col) for col in cols_to_remove]

print("Dropping indices:", drop_indices)

# Sort descending so deleting works without shifting indices
drop_indices_sorted = sorted(drop_indices, reverse=True)

for idx in drop_indices_sorted:
    X_val   = np.delete(X_val,   idx, axis=1)
    X_test  = np.delete(X_test,  idx, axis=1)

    # Remove from feature column names
    feature_columns.pop(idx)


Dropping indices: [3, 4, 2]


In [3]:
from catboost import CatBoostClassifier

model = CatBoostClassifier()
model.load_model("models/catboost_model_optimized_parameters2.cbm")


In [4]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score

# Probabilities
y_val_pred_proba = model.predict_proba(X_val)[:, 1]
y_test_pred_proba = model.predict_proba(X_test)[:, 1]

# Convert to class predictions
y_val_pred = (y_val_pred_proba > 0.5).astype(int)
y_test_pred = (y_test_pred_proba > 0.5).astype(int)

print("VAL metrics:")
print("Accuracy:", accuracy_score(y_val, y_val_pred))
print("Precision:", precision_score(y_val, y_val_pred))
print("Recall:", recall_score(y_val, y_val_pred))
print("F1:", f1_score(y_val, y_val_pred))
print("AUC:", roc_auc_score(y_val, y_val_pred_proba))


print("\nTEST metrics:")
print("Accuracy:", accuracy_score(y_test, y_test_pred))
print("Precision:", precision_score(y_test, y_test_pred))
print("Recall:", recall_score(y_test, y_test_pred))
print("F1:", f1_score(y_test, y_test_pred))
print("AUC:", roc_auc_score(y_test, y_test_pred_proba))

VAL metrics:
Accuracy: 0.945772133081261
Precision: 0.6844085426579617
Recall: 0.9246524144117207
F1: 0.7865954470303955
AUC: 0.9824789227907951

TEST metrics:
Accuracy: 0.9436875068673772
Precision: 0.6739913212076447
Recall: 0.9276909391282246
F1: 0.7807486631016043
AUC: 0.9821804372552597
